In [ ]:
def is_psd(matrix, tol=1e-8):
    """
    判断一个矩阵是否是半正定 (PSD) 的。

    参数:
    - matrix (np.ndarray): 要检查的矩阵。
    - tol (float): 允许的误差容限，特征值小于此值视为0。

    返回:
    - bool: 如果矩阵是 PSD 的，返回 True；否则返回 False。
    """
    # 检查是否为方阵
    if not isinstance(matrix, np.ndarray) or matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Input must be a square matrix.")

    # 对称性检查
    if not np.allclose(matrix, matrix.T, atol=tol):
        return False

    # 计算特征值
    eigenvalues = np.linalg.eigvalsh(matrix)

    # 判断是否所有特征值均大于等于 -tol
    return np.all(eigenvalues >= -tol)

# 示例
#print("Is PSD:", is_psd(matrix))

In [ ]:
def near_psd(a, epsilon= 0.0):
    """
    修正矩阵为最近的半正定矩阵。
    """
    n = a.shape[0]
    inv_sd = None
    out = np.copy(a)

    # 如果是协方差矩阵，先转化为相关性矩阵
    if not np.allclose(np.diag(out), 1.0):
        inv_sd = np.diag(1.0 / np.sqrt(np.diag(out)))
        out = inv_sd @ out @ inv_sd

    # SVD 分解并修正特征值
    eigvals, eigvecs = np.linalg.eigh(out)
    eigvals = np.maximum(eigvals, epsilon)
    T = 1.0 / np.sqrt(np.sum(eigvecs ** 2 * eigvals, axis=1))
    T = np.diag(T)
    l = np.diag(np.sqrt(eigvals))
    B = T @ eigvecs @ l
    out = B @ B.T

    # 如果是协方差矩阵，将方差加回来
    if inv_sd is not None:
        inv_sd = np.diag(1.0 / np.diag(inv_sd))
        out = inv_sd @ out @ inv_sd

    return out

In [ ]:
import numpy as np
import sys

def get_Pu(A, W):
    Aret = A.copy()
    np.fill_diagonal(Aret, 1.0)
    return Aret

def get_Ps(A, W):
    W_sqrt = np.sqrt(W)
    W_sqrt_negative = np.linalg.inv(W_sqrt)
    n, k = A.shape
    temp = W_sqrt @ A @ W_sqrt
    lambda_, S = np.linalg.eigh(temp)
    temp_ = S @ np.diag(np.maximum(lambda_, 0)) @ S.T
    out = W_sqrt_negative @ temp_ @ W_sqrt_negative
    return out


def ga(Y, A):
    temp = Y - A
    # Ensure all elements in temp are numeric before squaring
    temp = temp.astype(float)  # Convert temp to numeric type
    out = np.sum(temp ** 2)
    return out

def higham_nearestPSD(A):

    n, k = A.shape
    Y = A.copy()
    invSD = None

    #calculate the correlation matrix if we got a covariance
    if not np.allclose(np.diag(Y), 1.0):
        invSD = np.diag(1 / np.sqrt(np.diag(Y)))
        Y = invSD @ Y @ invSD

    w = np.eye(n)
    S = 0
    iteration = 0
    gama = sys.float_info.max
    while iteration < 1e8:
        R = Y - S
        X = get_Ps(R, w)
        S = X - R
        Y = get_Pu(X, w)
        gama_ = ga(Y, A)
        minEigVal = np.min(np.linalg.eigvals(Y).real)
        if np.abs((gama_-gama)) < 1e-9 and minEigVal > -1e-8:
            break
        gama = gama_
        iteration += 1
    # eigenvalues, eigenvectors = np.linalg.eigh(Y)
    # epsilon = 1e-8
    # eigenvalues = np.maximum(eigenvalues, epsilon)  # 将小的负特征值变为一个正的 epsilon
    # Y = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
    #Add back the variance

    if invSD is not None:
        invSD = np.diag(1 / np.diag(invSD))
        Y = invSD @ Y @ invSD

    return Y

In [ ]:
def chol_psd(a):
    """
    对半正定矩阵进行 Cholesky 分解。
    """
    n = a.shape[0]
    root = np.zeros_like(a)

    for j in range(n):
        s = 0.0
        if j > 0:
            s = np.dot(root[j, :j], root[j, :j])

        temp = a[j, j] - s
        if 0 >= temp >= -1e-8:
            temp = 0.0
        root[j, j] = np.sqrt(temp)

        if root[j, j] == 0.0:
            root[j, j + 1:] = 0.0
        else:
            ir = 1.0 / root[j, j]
            for i in range(j + 1, n):
                s = np.dot(root[i, :j], root[j, :j])
                root[i, j] = (a[i, j] - s) * ir

    return root